1) Querying the associated genes to Dravet Syndrome 

In [ ]:
SELECT DISTINCT ?gene ?geneName
WHERE {
	?gda sio:SIO_000628 ?gene, ?disease .
	?gene rdf:type ncit:C16612 ;
		dcterms:title ?geneName .
	?disease rdf:type ncit:C7057 ;
		dcterms:title "Infantile Severe Myoclonic Epilepsy"@en .
}


In [ ]:
2) Querying the associated diseases to SCN1A

In [ ]:
SELECT DISTINCT ?disease ?diseaseName 
WHERE { 
  ?gda sio:SIO_000628 ?gene, ?disease . 
  ?gene rdf:type ncit:C16612 . 
  ?disease dcterms:title ?diseaseName . 
} 


In [ ]:
3) Overlap between gene and disease results 


In [ ]:
SELECT DISTINCT ?gene ?geneName ?disease ?diseaseName
WHERE {
	?gda sio:SIO_000628 ?gene, ?disease .
	?gene rdf:type ncit:C16612 ;
		dcterms:title ?geneName .
	?disease rdf:type ncit:C7057 ;
		dcterms:title "Infantile Severe Myoclonic Epilepsy"@en .
	?gda2 sio:SIO_000628 ?gene, ?disease2 .
	?disease2 dcterms:title ?diseaseName .
	FILTER (?disease != ?disease2)
}


In [ ]:
4) Find the protiens linked to these results 

In [ ]:
library(curl)
library(jsonlite)

# URL for STRING API query
query <- "SCN1A%0dDravet%20syndrome"
url <- paste0("https://string-db.org/api/json/get_string_ids?identifiers=", query, "&species=9606")

# API request
handle <- curl(url)
results <- readLines(handle)
close(handle)

json <- fromJSON(results)

# get protein IDs for SCN1A gene
scn1a_protein_ids <- json$stringId[json$preferredName == "SCN1A"]
print(scn1a_protein_ids)

# get protein IDs for Dravet syndrome
dravet_protein_ids <- json$stringId[json$preferredName == "Dravet syndrome"]
print(dravet_protein_ids)



In [ ]:
5) network construction of this information

In [ ]:
# protein nodes
proteins <- c("SCN1A", scn1a_protein_ids)
diseases<- c("Dravet syndrome", dravet_protein_ids)

# edges
edges <- matrix(c("SCN1A", scn1a_protein_ids), ncol = 2, byrow = TRUE)

# create graph 
g <- graph_from_edgelist(edges, directed = TRUE)

# Set node names
V(g)$label <- proteins

# node colors
V(g)$color <- ifelse(V(g)$label == "SCN1A", "red", "blue")

# edge colors
E(g)$color <- "black"
  
# plot network
plot(g, edge.curved = 0.2, edge.arrow.size = 0.5, layout = layout_nicely)


In [ ]:
6) Query drug targets from drugbank of the protien, gene and disease + network construction 


In [ ]:
library(curl)
library(jsonlite)
library(igraph)

# URL for STRING API query
query <- "SCN1A%0dDravet%20syndrome"
url <- paste0("https://string-db.org/api/json/get_string_ids?identifiers=", query, "&species=9606")

# API request to STRING
handle <- curl(url)
results <- readLines(handle)
close(handle)

json <- fromJSON(results)

#protein ID for SCN1A gene
scn1a_protein_id <- json$stringId[json$preferredName == "SCN1A"]

#protein ID for Dravet syndrome
dravet_protein_ids <- json$stringId[json$preferredName == "Dravet syndrome"]

#protein nodes
proteins <- c("SCN1A", scn1a_protein_id, dravet_protein_ids)
protein_names <- c("SCN1A", "SCN1A protein", "Dravet protein")
protein_colors <- c("red", "blue", "blue")

#edges
edges <- matrix(c("SCN1A", scn1a_protein_id, "SCN1A protein", dravet_protein_ids), ncol = 2, byrow = TRUE)

# Create graph 
g <- graph_from_edgelist(edges, directed = TRUE)

# node names and colors
V(g)$label <- protein_names
V(g)$color <- protein_colors

# edge colors
E(g)$color <- "black"
  
# drugBank API query
db_url <- "https://api.drugbank.com/v1/targets?search="

# API request to DrugBank
drug_targets <- list()
for (protein_id in proteins) {
  drugbank_url <- paste0(db_url, protein_id)
  handle <- curl(drugbank_url)
  results <- readLines(handle)
  close(handle)
  json <- fromJSON(results)
  if (length(json$targets) > 0) {
    drug_targets[[protein_id]] <- json$targets[[1]]$drugs
  } else {
    drug_targets[[protein_id]] <- NULL
  }
}

#  drug target nodes
drug_nodes <- c()
drug_names <- c()
for (protein_id in names(drug_targets)) {
  if (!is.null(drug_targets[[protein_id]])) {
    drug_nodes <- c(drug_nodes, paste0(protein_id, "-", drug_targets[[protein_id]]$drugbank_id))
    drug_names <- c(drug_names, drug_targets[[protein_id]]$name)
  }
}

# add drug target nodes to graph
g <- add.vertices(g, length(drug_nodes), label = drug_names, color = "green")
V(g)$label <- c(protein_names, drug_names)

# add drug target edges to graph
drug_edges <- matrix(ncol = 2, nrow = 0)
for (protein_id in names(drug_targets)) {
  if (!is.null(drug_targets[[protein_id]])) {
    for (i in 1:length(drug_targets[[protein_id]]$drugbank_id)) {
      drug_edges <- rbind(drug_edges, c(paste0(protein_id, "-", drug_targets[[protein_id]]$drugbank_id[i]), protein_id))
    }
  }
}
g <- add.edges(g, drug_edges)

# plot network
plot(g, edge.curved = 0.2, edge.arrow.size = 0.5, layout = layout_nicely)
